# LangChain Expression Language (LCEL)

A Linguagem de Expressão LangChain, ou LCEL, é uma forma declarativa de **compor cadeias de maneira fácil**. A LCEL foi projetada desde o primeiro dia para suportar a colocação de protótipos em produção, sem a necessidade de alterações no código, desde a cadeia mais simples “prompt + LLM” até as cadeias mais complexas (já vimos pessoas executando com sucesso cadeias LCEL com centenas de etapas em produção). Para destacar alguns dos motivos pelos quais você pode querer usar a LCEL:
- **Suporte a streaming de primeira classe**: menor tempo possível para saída do primeiro token produzidio;
- **Suporte assíncrono**: Qualquer cadeia construída com a LCEL pode ser chamada tanto com a API síncrona;
- **Execução paralela otimizada**: Sempre que suas cadeias LCEL tiverem etapas que podem ser executadas em paralelo, automaticamente é feito isso;
- **Retentativas e fallbacks**: É maneira de tornar suas cadeias mais confiáveis em grande escala, na qual ações alternativas podem ser tomadas no caso de um erro em uma cadeia
- **Acessar resultados intermediários**: auxiliando na depuração de uma cadeia;

## Um exemplo simples de LCEL

O exemplo mais simples que podemos mostrar seria na construção de *prompt + model*

In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Create a sentence about the following topic: {topic}')

In [7]:
chain = prompt | model

In [8]:
chain.invoke({'topic': 'kitten'})

AIMessage(content='The playful kitten pounced on the colorful yarn with excitement.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 16, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1dfd79ca-4bfc-48ff-acf1-6cba4ff3bdad-0')

## Adicionando mais elementos a chain

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [10]:
chain.invoke({'topic': 'kitten'})

'The playful kitten chased after the toy mouse with boundless energy.'

### Para não errar a ordem

É importante entendermos que cada componente recebe um tipo de entrada e gera um tipo de saída, e estes tipos precisam casar:

| Component       | Tipo de Entrada                                    | Tipo de Saída        |
|-----------------|-----------------------------------------------------|----------------------|
| Prompt          | Dicionário                                         | PromptValue      |
| ChatModel       | String única, lista de mensagens de chat ou PromptValue | Mensagem de Chat     |
| LLM             | String única, lista de mensagens de chat ou PromptValue | String               |
| OutputParser    | A saída de um LLM ou ChatModel                     | Depende do parser    |
| Retriever       | String única                                       | Lista de Documentos  |
| Tool            | String única ou dicionário, dependendo da ferramenta| Depende da ferramenta|

Podemos verificar isso pelos output e input schemas:

In [11]:
print(prompt.input_schema.schema_json(indent=4))

{
    "title": "PromptInput",
    "type": "object",
    "properties": {
        "topic": {
            "title": "Topic",
            "type": "string"
        }
    }
}


In [12]:
chain = prompt | model | output_parser
print(chain.input_schema.schema_json(indent=4))

{
    "title": "PromptInput",
    "type": "object",
    "properties": {
        "topic": {
            "title": "Topic",
            "type": "string"
        }
    }
}


### Como fazer com chains clássicas?

In [15]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("Create a sentence about the following topic: {topic}")
output_parser = StrOutputParser()

In [16]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser
)
chain.invoke({'topic': 'kitten'})

{'topic': 'kitten',
 'text': 'The playful kitten pounced on the ball of yarn, batting it around with its tiny paws.'}